In [5]:
# !/usr/bin/env python
import pika
import json
credentials = pika.PlainCredentials('rabbitadmin','yulorei7')
connection = pika.BlockingConnection(pika.ConnectionParameters(
    '172.18.19.219',5672,'/',credentials))
channel = connection.channel()

# 声明queue
channel.queue_declare(queue='rpt_in_queue',durable=True)
import json
json_filename = ('../mongo/phone.json') 
_data=json.load(open(json_filename)) 

post_data=json.dumps({'data':_data,'cfg':{},'sid':'','retry_times':1})
# n RabbitMQ a message can never be sent directly to the queue, it always needs to go through an exchange.
channel.basic_publish(exchange='',
                      routing_key='rpt_in_queue',
                      body=post_data,
                      properties=pika.BasicProperties(
                          delivery_mode=2,  # make message persistent
                      )
                     )
print(" [x] Sent 'Hello World!'")
connection.close()

 [x] Sent 'Hello World!'


In [52]:
# !/usr/bin/env python
import pika
import json
credentials = pika.PlainCredentials('rabbitadmin','yulorei7')
connection = pika.BlockingConnection(pika.ConnectionParameters(
    '172.18.19.219',5672,'/',credentials))
channel = connection.channel()

exchangeName='dlx'
queueName='delayed_queue'
# 创建交换机
#channel.exchange_declare(exchange=exchangeName, exchange_type='topic', durable=True)
#channel.queue_declare(queue=queueName, durable=True,arguments={'x-message-ttl':30000})
#channel.queue_bind(queue=queueName, exchange=exchangeName, routing_key="dlx")

# 声明queue
#arguments={'x-message-ttl' : 900000}
#channel.queue_declare(queue='delayed_c_in_queue',durable=True)
post_data=json.dumps({'data':'','cfg':{},'sid':'','retry_times':1})

# n RabbitMQ a message can never be sent directly to the queue, it always needs to go through an exchange.
channel.basic_publish(exchange=exchangeName,
                      routing_key='dlx',
                      body=post_data,
                      properties=pika.BasicProperties(
                          delivery_mode=2,  # make message persistent   
                    content_type='application/json',
                    content_encoding='UTF-8',
                    priority=0,
                    expiration="30000",
                      )
                     )
print(" [x] Sent 'Hello World!'")
connection.close()

 [x] Sent 'Hello World!'
